# Comparison without classification

In [2]:
import torch
from torch.utils.data import Dataset
import torch.nn.functional as F
import torch.nn as nn
import numpy as np

## Make Data

In [47]:
class conComparisonData(torch.utils.data.Dataset):
    
    def __init__(self):
        #unnecessary because they are all the same
        self.x = torch.zeros([500,2])
        self.y = torch.zeros([500,1])
        k = 0
        for i in range(1,6):
            for j in range(50):
                self.x[k][0] = i-1
                self.x[k][1] = i
                self.y[k][0] = 0
                k += 1
                self.x[k][0] = i
                self.x[k][1] = i-1
                self.y[k][0] = 1
                k += 1
        self.len = len(self.x)
            
    def __getitem__(self, index):
        return (self.x[index], self.y[index])
    
    def __len__(self):
        return self.len
        

In [49]:
dataset = conComparisonData()
len(dataset)
dataset[0]

(tensor([0., 1.]), tensor([0.]))

## Model

In [6]:
class Model(nn.Module):
    def __init__(self,ind,h1,outd):
        super(Model, self).__init__()
        self.lin1 = nn.Linear(ind,h1)
        self.lin2 = nn.Linear(h1,outd)
    
    def forward(self,x):
        x = torch.relu(self.lin1(x))
        x = torch.sigmoid(self.lin2(x))
        return x

model = Model(2,4,1)
model.state_dict()

OrderedDict([('lin1.weight',
              tensor([[ 0.6005, -0.3679],
                      [-0.4366,  0.3920],
                      [ 0.0515,  0.1376],
                      [ 0.6551, -0.4528]])),
             ('lin1.bias', tensor([-0.5528,  0.1071, -0.1619, -0.0622])),
             ('lin2.weight', tensor([[ 0.0783,  0.1616,  0.3737, -0.4999]])),
             ('lin2.bias', tensor([0.1046]))])

In [7]:
x = dataset[0][0]
print(x)
yhat = model(x)
yhat
len(model(dataset.x))
dataset.y.shape

tensor([0., 1.])


torch.Size([500, 1])

## Training and Validation

In [8]:
def train(model,criterion,optimizer,dataset):
    lossList = []
    for i in range(10000):
        optimizer.zero_grad()
        yhat = model(dataset.x)
        #print(yhat)
        loss = criterion(yhat,dataset.y)
        lossList.append(loss.item())
        loss.backward()
        optimizer.step()
        if i % 1000 == 0:
            print('epoch ', i, ' loss: ', str(loss.item()))
    return lossList    
    

In [23]:
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
lossL = train(model,criterion,optimizer,dataset)

epoch  0  loss:  0.0023376168683171272
epoch  1000  loss:  0.002042541280388832
epoch  2000  loss:  0.0018085803603753448
epoch  3000  loss:  0.001619085669517517
epoch  4000  loss:  0.0014627992641180754
epoch  5000  loss:  0.0013319465797394514
epoch  6000  loss:  0.0012209804262965918
epoch  7000  loss:  0.0011257779551669955
epoch  8000  loss:  0.0010433244751766324
epoch  9000  loss:  0.0009713053586892784


In [24]:
vL = torch.tensor([[0.,1.],[1.,2.],[2.,3.],[3.,4.],[4.,5.]])
v0 = model(vL)
v0

tensor([[0.0044],
        [0.0005],
        [0.0004],
        [0.0004],
        [0.0005]], grad_fn=<SigmoidBackward>)

In [25]:
vG = torch.tensor([[1.,0.],[2.,1.],[3.,2.],[4.,2.],[5.,4.]])
v1 = model(vG)
v1

tensor([[0.9992],
        [0.9993],
        [0.9995],
        [1.0000],
        [0.9996]], grad_fn=<SigmoidBackward>)

## Test with untrained comparisons

In [3]:
testing0 = torch.zeros([10,2])
k = 0
for i in range(4):
    for j in range(i+2,6):
        testing0[k][0] = i
        testing0[k][1] = j
        k += 1
print(testing0)
testing1 = torch.zeros([10,2])
k = 0
for i in range(5,-1,-1):
    for j in range(i-2,-1,-1):
        testing1[k][0] = i
        testing1[k][1] = j
        k += 1
#print(testing1)

tensor([[0., 2.],
        [0., 3.],
        [0., 4.],
        [0., 5.],
        [1., 3.],
        [1., 4.],
        [1., 5.],
        [2., 4.],
        [2., 5.],
        [3., 5.]])


In [44]:
result0 = model(testing0)
result0

tensor([[2.5940e-04],
        [1.5220e-05],
        [8.9280e-07],
        [5.2371e-08],
        [3.0439e-05],
        [1.7856e-06],
        [1.0474e-07],
        [3.5711e-06],
        [2.0948e-07],
        [4.1894e-07]], grad_fn=<SigmoidBackward>)

In [45]:
result1 = model(testing1)
result1

tensor([[1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000],
        [1.0000]], grad_fn=<SigmoidBackward>)